# 單位 Function
---
![單位 Function](./imgs/Identity Function.png)
根據輸入的位置  
用 Hidden unit 壓縮成數字編碼  
在輸出

# Import lib
---

In [1]:
import tensorflow as tf
import numpy as np
import random

# 建構網路
---

In [2]:
# 要輸入幾個
vectorSize = 8

# 壓縮二進制
hiddenUnits = 2

In [3]:
# 輸入
inputValue = tf.placeholder(
    tf.float32,
    [None, vectorSize],
    name="InputValue"
)

# label 後的值
labelValue = tf.placeholder(
    tf.float32,
    [None, vectorSize],
    name="LabelValue"
)

In [4]:
weightInit = tf.random_normal_initializer(mean=0, stddev=0.3)
biasInit = tf.random_normal_initializer(mean=0, stddev=0.1)

In [5]:
hiddenlayer = tf.layers.dense(
    inputs=inputValue,
    units=hiddenUnits,
    activation=tf.nn.sigmoid,
    kernel_initializer=weightInit,
    bias_initializer= biasInit,
    name="HiddenLayer"
)

output = tf.layers.dense(
    inputs=hiddenlayer,
    units=vectorSize,
    kernel_initializer=weightInit,
    bias_initializer=biasInit,
    name="Output"
)

outputProb = tf.nn.softmax(output, name="OutputProb")

predValue = tf.argmax(outputProb, axis=1)

In [6]:
with tf.name_scope("Trainning"):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=labelValue)
    lossMean = tf.reduce_mean(loss)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-1).minimize(lossMean)

In [7]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [8]:
# logFile = tf.summary.FileWriter("./logs")
# logFile.add_graph(session.graph)

## 建 Training Data
---

In [9]:
# 所有可能的 DataSet
dataset = [
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]
            ]

In [10]:
# 測試資料的筆數
trainNum = 10000

In [11]:
# 產生 DataIndex & 產生 Train Set (trainNum)
trainSet = []
for i in range(0, trainNum):
    chooseIndex = random.randint(0, len(dataset) - 1)
    
    # 將資料
    trainSet.append(dataset[chooseIndex])

## Session 建構並開始學習
---

In [12]:
def printPred():
    for i in range(0, vectorSize):
        vInput = np.argmax(dataset[i])
        vHidden = session.run(hiddenlayer, feed_dict={inputValue: [dataset[i]]})
        vOutput = session.run(predValue, feed_dict={inputValue: [dataset[i]]})[0]
        print(format(vInput) + " => " + format(vHidden) + " => " + format(vOutput))

In [13]:
printPred()

0 => [[ 0.43426651  0.50617534]] => 4
1 => [[ 0.58396679  0.42753848]] => 4
2 => [[ 0.42673478  0.41185164]] => 4
3 => [[ 0.45913541  0.38561335]] => 4
4 => [[ 0.53627789  0.56031305]] => 4
5 => [[ 0.52142668  0.47002977]] => 4
6 => [[ 0.453217    0.46424437]] => 4
7 => [[ 0.53325588  0.39084435]] => 4


In [14]:
for i in range(0, trainNum, 10):
    feedDict = {inputValue: trainSet[i: i+10], labelValue: trainSet[i: i+10]}
    session.run(optimizer, feed_dict=feedDict)

## 看結果確率
---

In [15]:
printPred()

0 => [[  9.99256790e-01   4.43048863e-04]] => 0
1 => [[ 0.15175976  0.99877149]] => 1
2 => [[ 0.00013926  0.00047852]] => 2
3 => [[  3.59210968e-01   1.50915250e-04]] => 3
4 => [[ 0.81105113  0.99975687]] => 4
5 => [[ 0.99956769  0.43430796]] => 5
6 => [[ 0.32883087  0.35754547]] => 6
7 => [[  1.53849658e-04   4.39515918e-01]] => 7


# 結論
---
這個問題可能藉由更低的數字來逼近解  
因為在 hidden layer 的值  
可能也會成為分類的根據